#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.
 
cleaning London airbnb file

v.1.3. 2021-05-17 paths changed


in: data from web

out: airbnb_london_cleaned.csv

In [1]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore")

In [2]:
print(os.getcwd()) 

/Users/ghadena/Desktop/Business analytics/DA3/DA3/assignment_1


In [3]:
data_dir = "/Users/ghadena/Desktop/Business analytics/DA3/DA3/assignment_1/"

# location of folders
data_in = data_dir + "data/raw/"
data_out = data_dir + "data/clean/"

In [4]:
# zero step
data = pd.read_csv(data_in + "listings.csv.gz", delimiter=",", dtype="unicode")
drops = [
    "host_thumbnail_url",
    "host_picture_url",
    "listing_url",
    "picture_url",
    "host_url",
    "last_scraped",
    "description",
    "neighborhood_overview",
    "host_about",
    "host_response_time",
    "name",
    "host_location",
]
data.drop(columns=drops, inplace=True)
data.to_csv(data_in + "airbnb_madrid_listing.csv", index=False)

In [5]:
# opening dataset
df = pd.read_csv(data_in + "airbnb_madrid_listing.csv", delimiter=",")

In [6]:

# Step 1: Identify the Review Columns and Add the 'accommodates' Column
review_columns = [col for col in df.columns if 'review' in col.lower()]
columns_to_extract = review_columns + ['accommodates']
print(f"Columns to extract: {columns_to_extract}")

# Step 2: Extract the Columns from the Original DataFrame
review_df = df[columns_to_extract].copy()

Columns to extract: ['number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'accommodates']


In [7]:
# drop broken lines - where id is not a character of numbers
df.id = pd.to_numeric(df.id, errors="coerce")
df = df[df.id.notna()]

In [8]:
# display the class and type of each columns
df.dtypes

id                                                int64
scrape_id                                         int64
source                                           object
host_id                                           int64
host_name                                        object
                                                 ...   
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
Length: 63, dtype: object

In [9]:
#####################
# formatting columns
for perc in ["host_response_rate", "host_acceptance_rate"]:
    df[perc] = pd.to_numeric(df[perc], errors="coerce")

In [10]:
#remove dollar sign from price
if 'price' in df.columns:
    df['price'] = df['price'].dropna().astype(str).str.replace(r"\$", "", regex=True)
    df['price'] = pd.to_numeric(df['price'], errors="coerce")

# Check results
print(df['price'].head(10))
print(df['price'].isnull().sum())  # Count of remaining null values
print(df['price'].dtype)  # Should be float64

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8    150.0
9      NaN
Name: price, dtype: float64
6018
float64


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26980 entries, 0 to 26979
Data columns (total 63 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            26980 non-null  int64  
 1   scrape_id                                     26980 non-null  int64  
 2   source                                        26980 non-null  object 
 3   host_id                                       26980 non-null  int64  
 4   host_name                                     26977 non-null  object 
 5   host_since                                    26977 non-null  object 
 6   host_response_rate                            0 non-null      float64
 7   host_acceptance_rate                          0 non-null      float64
 8   host_is_superhost                             26001 non-null  object 
 9   host_neighbourhood                            9401 non-null  

In [12]:
# format binary variables
for binary in [
    "host_is_superhost",
    "host_has_profile_pic",
    "host_identity_verified",
    "instant_bookable",
]:
    df[binary] = df[binary].map({"t": True, "f": False})

In [13]:
# amenities
df["amenities"] = df["amenities"].str.strip("{}").str.replace('"', "").str.split(",")

In [14]:
#generate dummies by amenities

# # Generate dummies from amenities column
# dummies = pd.get_dummies(df["amenities"].apply(pd.Series).stack()).groupby(level=0).sum()

# # Concatenate the new dummies with the original DataFrame
# df = pd.concat([df, dummies], axis=1)

# df.drop(columns=[
#     "", 
#     "amenities", 
#     "translation missing: en.hosting_amenity_49", 
#     "translation missing: en.hosting_amenity_50"
# ], inplace=True, errors="ignore")

# # Check result
# print(df.head())
# print(sorted(df["amenities"].explode().unique()))
# # dummies = pd.get_dummies(df.amenities.apply(pd.Series).stack()).sum(level=0)
# # df = pd.concat([df, dummies], axis=1)


# # drops = [
# #     "",
# #     #"amenities",
# #     "translation missing: en.hosting_amenity_49",
# #     "translation missing: en.hosting_amenity_50",
# # ]
# # df.drop(columns=drops, inplace=True)

In [15]:

# Standardize amenities before creating dummies
df["amenities"] = (
    df["amenities"]
    .astype(str)  # Ensure it's a string
    .str.lower()  # Convert to lowercase for consistency
    .str.replace(r"[\[\]\{\}]", "", regex=True)  # Remove brackets
    .str.replace(r"\s+", " ", regex=True)  # Remove extra spaces
    .str.strip()  # Trim spaces at the beginning/end
)

# Convert amenities into separate columns
dummies = pd.get_dummies(df["amenities"].apply(lambda x: pd.Series(x.split(","))).stack()).groupby(level=0).sum()

# Concatenate with original DataFrame
df = pd.concat([df, dummies], axis=1)

# Drop the original 'amenities' column
df.drop(columns=["amenities"], inplace=True, errors="ignore")

In [16]:
# Check results
print(df.columns)

Index(['id', 'scrape_id', 'source', 'host_id', 'host_name', 'host_since',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood',
       ...
       ''toaster'', ''tv with amazon prime video'', ''tv with dvd player'',
       ''tv with hbo max'', ''tv'', ''varias body soap'',
       ''washer \\u2013\\u00a0in unit'', ''washer'', ''wifi'',
       ''wine glasses''],
      dtype='object', length=3218)


In [17]:
df.columns.tolist()

['id',
 'scrape_id',
 'source',
 'host_id',
 'host_name',
 'host_since',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'calendar_last_scraped',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'first_review',
 'last_review',
 'review_scores_rating',
 're

In [18]:
import pandas as pd
import re

def clean_column_name(col):
    # Remove D_ prefix and any quotes/backslashes
    cleaned = re.sub(r'^d_[\'"]?|[\'"]$', '', col, flags=re.IGNORECASE)
    
    # Decode Unicode characters and special symbols
    cleaned = re.sub(r'\\u[0-9a-f]{4}', lambda m: bytes(m.group(0), 'utf-8').decode('unicode_escape'), cleaned)
    cleaned = re.sub(r'[^a-zA-Z0-9]', ' ', cleaned)  # Replace special chars with spaces
    
    # Standardize text
    cleaned = cleaned.lower().strip()
    cleaned = re.sub(r'\s+', '_', cleaned)  # Convert spaces to underscores
    
    return cleaned

# Apply to all columns
cleaned_columns = [clean_column_name(col) for col in df.columns]
df.columns = cleaned_columns 

In [19]:
aggregation_rules = {
    'shampoo': ['shampoo'],
    'conditioner': ['conditioner'],
    'body_soap': ['body_soap'],
    'tv': ['tv', 'hdtv', 'television'],
    'streaming_services': ['netflix', 'amazon_prime', 'hulu', 'disney+', 'streaming','appletv', 'chromecast','hbomax'],
    'refrigerator': ['refrigerator', 'fridge'],
    'stove': ['stove'],
    'child_friendly': ['child', 'baby', 'infant', 'toddler', 'high_chair', 'crib', 'babybath', 'babymonitor', 'changing_table', 'children', 'kids', 'family'],
    'free_parking': ['free_parking', 'freeresidentialgarageonpremises', 'freestreetparking', 'freeparkingonpremises'],
    'paid_parking': ['paid_parking', 'paidparkingoffpremises','paidparkingonpremises','paid_garage'],
    'wifi': ['wifi', 'internet'],
    'bidet': ['bidet'],
    'oven': ['oven'],
    'cable': ['cable'],
    'sound_system': ['sound_system', 'speaker'],
    'backyard': ['backyard', 'garden', 'yard'],
    'view': ['view'],
    'balcony': ['balcony'],
    'bbq': ['bbq'],
    'bathtub': ['bathtub'],
    'coffee_maker': ['coffee', 'espresso', 'keurig'],
    'exercise_equipment': ['exercise', 'free_weights', 'treadmill', 'elliptical', 'yoga', 'pilates','workout_bench','stationary_bike'],
    'dryer': ['dryer', 'free_dryer'],
    'washer': ['washer', 'free_washer'],
    'gym': ['gym'],
    'heating': ['heating','heated','central_heating'],
    'housekeeping_included': ['housekeeping', 'included', 'housekeeping_available'],
    'housekeeping_extracost': ['housekeeping', 'extra_cost','housekeeping_available'],
    'indoor_fireplace': ['indoor_fireplace'],
    'paid_dryer_washer': ['paid_washer', 'paid_dryer','laundromat'],
    'airconditioning': ['air_conditioning','central_airconditioning'],
    'kitchen': ['kitchen', 'kitchenette'],
    'game_console': ['game_console', 'playstation', 'xbox'],
    'clothing_storage': ['clothing_storage', 'closet', 'wardrobe', 'dresser'],
    'electric_car_charging': ['ev_charger', 'free_carport_on_premises'],
    'indoor_pool': ['indoor_pool'],
    'outdoor_pool': ['outdoor_pool','infinity','olympic_sized','lap_pool','pool_heathed'],
    'outdoor_space': ['outdoor_dining','outdoor_seating','outdoor_shower','outdoor_grill','outdoor_patio','outdoor_firepit','outdoor_bar','outdoor_barbecue','outdoor_furniture','outdoor_play'],
    'private_ameneties': ['private'],
    'shared_ameneties': ['shared'],
    'private_pool': ['private_pool'],
    'sauna': ['sauna','sauna_steam_room','private_sauna'], 
    # Add more rules as needed...
}

In [20]:
import re

def aggregate_features(df, rules, strict_multi_word=None):
    """
    Aggregates binary features based on keyword matching in column names.
    
    - For `strict_multi_word` categories (e.g., housekeeping), ALL words must be present in the column name.
    - For everything else, a column matches if ANY keyword appears.
    
    Args:
        df (DataFrame): The dataset with binary dummy variables.
        rules (dict): Mapping of new feature names to keyword lists.
        strict_multi_word (list): List of category names that require ALL keywords in the column name.
    
    Returns:
        DataFrame: Updated dataset with aggregated features.
    """
    strict_multi_word = strict_multi_word or []  # Default to an empty list if None
    columns_to_drop = []  # List to track original columns for removal

    for new_col, keywords in rules.items():
        # Escape special regex characters in keywords
        patterns = [re.escape(k) for k in keywords]
        
        if new_col in strict_multi_word:
            # 🟢 Strict Mode: Ensure ALL keywords appear together (for housekeeping-related columns)
            matching_cols = [
                col for col in df.columns
                if all(re.search(p, col, flags=re.IGNORECASE) for p in patterns)
            ]
        else:
            # 🔵 Standard Mode: Match if ANY keyword appears (for all other features)
            pattern = r'(' + '|'.join(patterns) + r')'
            matching_cols = [
                col for col in df.columns
                if re.search(pattern, col, flags=re.IGNORECASE)
            ]
        
        print(f"\nCreating '{new_col}' from:")
        print(matching_cols)
        
        # Create new aggregated column
        if matching_cols:
            df[new_col] = df[matching_cols].any(axis=1).astype(int)
            columns_to_drop.extend(matching_cols)  # ✅ Track columns to remove
    
    print(f"Columns before dropping: {df.columns.tolist()}")
    
    # ✅ Drop the original detailed columns that were aggregated
    # ✅ FIX: Prevent Aggregated Columns from Being Dropped
    columns_to_drop = [col for col in columns_to_drop if col not in rules.keys()]
    df.drop(columns=columns_to_drop, inplace=True, errors="ignore")

    print(f"Final columns after dropping: {df.columns.tolist()}")
    
    return df

# 🔵 Housekeeping Columns Require **Both Words**
strict_categories = ['housekeeping_extra_cost', 'housekeeping_included']

# 🚀 Apply Aggregation
df = aggregate_features(df, aggregation_rules, strict_categories)

print(df.info())  # Shows data types and missing values
print(df.describe())  # Shows summary statistics for numeric columns


Creating 'shampoo' from:
['elvive_de_l_oreal_shampoo', 'johnson_s_baby_shampoo', 'l_occitane_en_provence_shampoo', 'l_or_al_primeras_marcas_shampoo', 'l_or_al_shampoo', 'shampoo', 'shampoo', '3_marcas_distintas_shampoo', 'shampoo', 'shampoo', 'a_demanda_shampoo', 'acondicionador_y_gel_de_ducha_shampoo', 'aesop_shampoo', 'almendra_almonds_shampoo', 'almond_shampoo', 'aloe_vera_shampoo', 'alphaparf_shampoo', 'alqvimia_shampoo', 'amenitie_shampoo', 'amenities_de_bienvenida_shampoo', 'amenities_de_bienvenida_shampoo', 'amenities_shampoo', 'amenity_shampoo', 'any_shampoo', 'anyah_shampoo', 'apivita_shampoo', 'aussie_shampoo', 'avellana_shampoo', 'avon_shampoo', 'azl_shampoo', 'b_sico_shampoo', 'babaria_de_coco_shampoo', 'basico_shampoo', 'bienvenue_shampoo', 'bio_shampoo', 'blanc_shampoo', 'blanca_shampoo', 'blancas_shampoo', 'bodyplus_shampoo', 'bonte_shampoo', 'bote_de_dos_o_tres_dosis_shampoo', 'camomila_shampoo', 'carrefour_shampoo', 'cela_d_pend_shampoo', 'champ_de_intensa_hidrataci_n

In [21]:
df.columns.to_list()

['id',
 'scrape_id',
 'source',
 'host_id',
 'host_name',
 'host_since',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'calendar_last_scraped',
 'license',
 'instant_bookable',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 '',
 'contiene_alime

In [22]:
df.drop(columns=[''], inplace=True)



In [23]:
# write csv
#df.to_csv(data_out + "airbnb_madrid_cleaned.csv", index=False)

In [24]:
df.view.value_counts()
#df.airconditioning.value_counts()

view
1    21420
0     5560
Name: count, dtype: int64

In [25]:
import pandas as pd

# Assuming df is your original DataFrame


# Step 3: Merge the Extracted Columns Back into the Cleaned DataFrame
df = pd.concat([df, review_df], axis=1)

print("Merged the review columns and 'accommodates' column back into the cleaned DataFrame.")

Merged the review columns and 'accommodates' column back into the cleaned DataFrame.


In [26]:
import numpy as np
# Function to check if an element is multidimensional
def is_multidimensional(x):
    return isinstance(x, (list, tuple, np.ndarray))

# Apply the function to each element in the DataFrame
multidimensional_columns = df.applymap(is_multidimensional).any()

# Print columns with multidimensional data
print("Columns with multidimensional data:")
print(multidimensional_columns[multidimensional_columns].index.tolist())

# Inspect the dimensions of the multidimensional columns
for col in multidimensional_columns[multidimensional_columns].index:
    print(f"Column '{col}' dimensions:")
    print(df[col].apply(lambda x: np.array(x).shape if is_multidimensional(x) else (1,)).value_counts())

Columns with multidimensional data:
[]


In [27]:
import pandas as pd

# Assuming df is your original DataFrame

# Identify duplicated columns
duplicate_columns = df.columns[df.columns.duplicated(keep='first')]

# Drop the first instance of each duplicated column
df = df.loc[:, ~df.columns.duplicated(keep='first')]

print(f"Dropped the first instance of each duplicate column: {duplicate_columns.tolist()}")

Dropped the first instance of each duplicate column: ['body_soap', 'body_soap', 'shampoo', 'refrigerator', 'refrigerator', 'body_soap', 'shampoo', 'conditioner', 'refrigerator', 'body_soap', 'conditioner', 'shampoo', 'body_soap', 'conditioner', 'refrigerator', 'shampoo', 'backyard', 'baking_sheet', 'books_and_reading_material', 'breakfast', 'carbon_monoxide_alarm', 'cleaning_products', 'dedicated_workspace', 'dining_table', 'dishes_and_silverware', 'dryer', 'essentials', 'exercise_equipment', 'exterior_security_cameras_on_property', 'extra_pillows_and_blankets', 'fire_extinguisher', 'first_aid_kit', 'free_street_parking', 'freezer', 'hangers', 'heating', 'hockey_rink', 'host_greets_you', 'hot_tub', 'hot_water_kettle', 'hot_water', 'indoor_fireplace', 'iron', 'keypad', 'kitchen', 'lock_on_bedroom_door', 'long_term_stays_allowed', 'microwave', 'noise_decibel_monitors_on_property', 'paid_resort_access', 'pets_allowed', 'ping_pong_table', 'pool', 'refrigerator', 'self_check_in', 'shampoo',

In [28]:
df.shampoo

0        0
1        0
2        1
3        0
4        0
        ..
26975    0
26976    0
26977    0
26978    0
26979    1
Name: shampoo, Length: 26980, dtype: int64

In [31]:
# Identify duplicated columns
duplicate_columns = df.columns[df.columns.duplicated()]
duplicate_columns

Index([], dtype='object')

In [32]:
# Filter columns that contain the word "review"
review_columns = [col for col in df.columns if 'review' in col.lower()]

print("Columns that might contain review data:")
print(review_columns)

Columns that might contain review data:
['number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']


In [33]:
df.shape

(26980, 268)

In [34]:
df['accommodates']

0        4
1        1
2        2
3        2
4        2
        ..
26975    2
26976    3
26977    2
26978    2
26979    4
Name: accommodates, Length: 26980, dtype: int64

In [35]:
print(df.view.value_counts())
print(df.airconditioning.value_counts())


view
1    21420
0     5560
Name: count, dtype: int64
airconditioning
1    16124
0    10856
Name: count, dtype: int64


In [36]:
df.shape

(26980, 268)

In [37]:
df.to_csv(data_out + "testing.csv", index=False)